In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# --- PASSO 0: DEFINIÇÕES INICIAIS E PARÂMETROS ---
# Parâmetros fornecidos para guiar a geração de dados
FREQ_PARCIAL_ALVO = 0.03   # 3%
FREQ_TOTAL_ALVO = 0.001  # 0.1%
SEV_PARCIAL_ALVO = 3_000 # R$ 3.000

# Parâmetros para a simulação
N_APOLICES = 1000000
np.random.seed(42) # Para reprodutibilidade dos resultados

In [ ]:
# --- PASSO 1: CRIAÇÃO DO DATASET SINTÉTICO ---
print(f"Gerando um dataset sintético com {N_APOLICES} apólices...")

# 1.1. Criação das features preditivas (características do risco)
df = pd.DataFrame({
    'exposicao': np.random.uniform(0.1, 1.0, N_APOLICES), # Exposição entre 1.2 meses e 1 ano
    'classe_bonus': np.random.randint(0, 11, N_APOLICES),
    'idade': np.random.randint(18, 81, N_APOLICES),
    'rns': np.random.randint(0, 2, N_APOLICES), # 0 = sem RNS, 1 = com RNS
    'valor_veiculo': np.random.uniform(20000, 150001, N_APOLICES).round(2)
})

print(f"Gerando um dataset sintético com dimensões: {df.shape}")

# 1.2. Criação do "fator de risco" individual para cada apólice
# Usamos uma combinação linear das features para simular o risco latente.
# Sinais dos coeficientes:
# - classe_bonus: negativo (maior bônus, menor risco)
# - idade: negativo (maior idade, menor risco)
# - rns: positivo (ter RNS, maior risco)
log_risco_base_freq = (
    -0.15 * df['classe_bonus']
    -0.02 * df['idade']
    + 1.0 * df['rns']
)

# 1.3. Geração das Quantidades de Sinistros (Frequência)
# Ajustamos o intercepto para que a média da frequência na carteira se aproxime dos alvos.

# Colisão Parcial
# Calibração do intercepto para atingir a frequência alvo
taxa_media_parcial = np.exp(log_risco_base_freq).mean()
intercepto_parcial = np.log(FREQ_PARCIAL_ALVO / taxa_media_parcial)
taxa_poisson_parcial = np.exp(intercepto_parcial + log_risco_base_freq) * df['exposicao']
df['qtd_colisao_parcial'] = np.random.poisson(taxa_poisson_parcial)

# Colisão Total
# Calibração do intercepto
taxa_media_total = np.exp(log_risco_base_freq).mean()
intercepto_total = np.log(FREQ_TOTAL_ALVO / taxa_media_total)
taxa_poisson_total = np.exp(intercepto_total + log_risco_base_freq) * df['exposicao']
df['qtd_colisao_total'] = np.random.poisson(taxa_poisson_total)


# 1.4. Geração da Severidade de Colisão Parcial (para quem teve sinistro)
# A severidade depende do risco e, principalmente, do valor do veículo.
log_risco_base_sev = (
    -0.01 * df['classe_bonus']  # Influência menor
    -0.005 * df['idade']       # Influência menor
    + 0.1 * df['rns']          # Influência pequena
    + 0.5 * np.log(df['valor_veiculo']) # Influência maior e não-linear
)

# Calibração do intercepto para atingir a severidade alvo
media_sev_base = np.exp(log_risco_base_sev[df['qtd_colisao_parcial'] > 0]).mean()
intercepto_sev = np.log(SEV_PARCIAL_ALVO / media_sev_base)

# Média da Gamma para cada apólice com sinistro
media_gamma = np.exp(intercepto_sev + log_risco_base_sev)

# Parâmetro de forma da Gamma (controla a variância/dispersão)
shape_gamma = 20.0
scale_gamma = media_gamma / shape_gamma

# Gerar valores da Gamma apenas para quem teve sinistro
severidade_valores = np.random.gamma(shape=shape_gamma, scale=scale_gamma)

# Atribuir a severidade ao DataFrame
df['valor_colisao_parcial'] = 0.0
mask_sinistro_parcial = df['qtd_colisao_parcial'] > 0
# O valor total do sinistro é a quantidade de eventos * a severidade média por evento
# Para simplificar, assumimos que a severidade gerada é o valor total para aquela apólice
df.loc[mask_sinistro_parcial, 'valor_colisao_parcial'] = \
    severidade_valores[mask_sinistro_parcial].round(2) * df.loc[mask_sinistro_parcial, 'qtd_colisao_parcial']

# Amostra para casos com colisão parcial
df.shape

In [ ]:
df.info()

In [ ]:
#write data to BQ
import pandas as pd
from google.cloud import bigquery

# ==============================================================================
# 1. CONFIGURE SUAS VARIÁVEIS
# ==============================================================================
# Substitua pelo ID do seu projeto GCP.
project_id = "project-poc-purple"

# Nome do dataset no BigQuery (deve existir)
dataset_id = "demos"

# Nome da tabela que será criada ou sobrescrita
table_id = "dados_apolices_v1"

# ==============================================================================
# 2. SEU DATAFRAME
# ==============================================================================
# Supondo que seu DataFrame de 1 milhão de linhas já está
# carregado na variável 'df'. Não é preciso criá-lo aqui.
# df = pd.read_csv(...) ou similar

# ==============================================================================
# 3. ENVIE OS DADOS PARA O BIGQUERY
# ==============================================================================
print("Iniciando o carregamento para o BigQuery...")

# Inicializa o cliente, especificando o projeto para evitar erros de quota.
client = bigquery.Client(project=project_id)

# Configura o job de carregamento.
# 'autodetect=True' é perfeito para o seu caso.
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE", # Sobrescreve a tabela. Mude para WRITE_APPEND se quiser adicionar.
    autodetect=True,
)

# Monta a referência completa da tabela.
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Executa o job. Esta função é eficiente para grandes DataFrames. 🚀
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Aguarda a finalização do job.
job.result()

print(f"✅ Sucesso! Os {len(df)} registros foram carregados na tabela {table_ref}.")

In [ ]:
#Ler dados da tabela do BQ:

print(f"Lendo dados da tabela {table_ref}...")
sql_query = f"""
    SELECT *
    FROM `{table_ref}`
"""
df = client.query(sql_query).to_dataframe()

print("✅ Sucesso! Dados carregados no DataFrame.")
print("-" * 30)
print("Informações do DataFrame carregado:")
df.info()
colunas_para_converter = df.select_dtypes(include='Int64').columns
for col in colunas_para_converter:
    df[col] = df[col].astype('int64')
df.info()